#  Iterative phase estimation

|||
|-|-|
|**Authors:** |Taha Selim, Alain Chancé|
|**Date:** |October 21, 2023|
|**Version:** |**1.00**<br/>*Details see at the end of this notebook*|
|**References:**|
[Prof. Gerrit C. Groenenboom, Quantum theoretical chemistry (NWI-MOL112), May 26, 2023](https://www.theochem.ru.nl/ctc2/pdf/qtclecture.pdf)
[Prof. Gerrit C. Groenenboom, Computational and Theoretical Chemistry 2, NWI-MOL176 (3EC)](https://www.theochem.ru.nl/ctc2/)
[Keeper L. Sharkey, Alain Chancé, Quantum Chemistry and Computing for the Curious: Illustrated with Python and Qiskit® code, ISBN-13: 978-1803243900](https://www.amazon.com/Quantum-Chemistry-Computing-Curious-Illustrated/dp/1803243902/)

The following MIT license only applies to the code, and not to the text and images.

# MIT License

Copyright (c) 2023 Taha Selim, Alain Chancé

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
using Plots # or StatsPlots
using LinearAlgebra
using SpecialFunctions
using AssociatedLegendrePolynomials
using LaTeXStrings
using Quantikz

## Include files
### Include lib_load.jl which loads common modules with includes

In [2]:
# import conventions
include("conventions.jl")
using .conventions: big_endian, qubit_begin, little_endian

# import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: q, Rz_gate1

include("lib_tensor/QTensor.jl")
using ..QTensor: q_T2D

include("lib_useful/custom_functions.jl")
using ..custom_functions: MK_sortrows

include("quantum_circuit.jl")
using ..quantum_circuit: qc_init, init_register, show_statevector, op
#using ..quantum_circuit: qc_init, init_register, print_initstate

Load quantum gates constructor


Load Tensor module: QTensor.jl
Load quantum gates constructor


Load quantum_circuit constructor
Load quantum_circuit constructor
Load quantum gates constructor
Load quantum_circuit constructor


# Qubits and quantum gates
## Qubits
A qubit is a quantum unit of information that represents a two-level quantum system and lives in a two-dimensional Hilbert space $\mathbb C^2$. The computational basis states of the quantum space are denoted as $\{|0\rangle,|1\rangle\}$:

$$|0\rangle=\left(\begin{array}{l}
1 \\
0
\end{array}\right) 
|1\rangle=\left(\begin{array}{l}
0 \\
1
\end{array}\right)$$

Any single-qubit state is described by a linear superposition of the computational basis with complex coefficients:  
$$|\psi\rangle=\alpha|0\rangle+\beta|1\rangle=\left(\begin{array}{l}
\alpha \\
\beta
\end{array}\right) \in \mathbb{C}^{2}$$

where 𝛼 and 𝛽 satisfy:
$$|\alpha|^{2}+|\beta|^{2}=1$$

A qubit is in a quantum superposition during the execution of an algorithm.  When it is measured in the computational basis, a qubit will be found in either state $|0\rangle$ or in state $|1\rangle$ with probability $|\alpha|^{2}$ and $|\beta|^{2}$ respectively. If there are $n$ qubits in the system, the state is described by a vector in the $2^n$ dimensional Hilbert space $(\mathbb{C}^2)^{⊗n}$ formed by taking the tensor product of the Hilbert spaces of the individual qubits. For example, for 10 qubits, the state is described by a vector in a 1024-dimensional Hilbert space. 

## Tensor ordering of qubits
The physics community typically orders a tensor product of $n$ qubits with the first qubit on the left-most side of the tensor product: 

$$|q\rangle=\left|q_{0}\right\rangle\left|q_{1}\right\rangle \ldots\left|q_{n-1}\right\rangle=\left|q_{0}, q_{1}, \ldots, q_{n-1}\right\rangle=\bigotimes_{i=0}^{n-1}\left|q_{i}\right\rangle$$

However, Qiskit uses an ordering in which the $n^{th}$ qubit is first in the order and the $0^{th}$ qubit is last:

$$|q\rangle=
\left|q_{n-1}\right\rangle\ldots\left|q_{1}\right\rangle\left|q_{0}\right\rangle
=\left|q_{n-1}, \ldots, q_{1}, q_{0}\right\rangle
=\bigotimes_{i=n-1}^{0}\left|q_{i}\right\rangle$$

In other words, if qubit $0$ is in state $|0\rangle$, qubit $1$ is in state $|0\rangle$, and qubit 2 is in state $|1\rangle$, the state represented in many physics textbooks as $|001\rangle$ is represented by Qiskit as $|100\rangle$. This difference affects the way multi-qubit operations are represented as matrices.

## Single qubit quantum gates
A single qubit quantum gate $U$ has a $(2\times2)$ unitary matrix form: $U^\dagger U=UU^\dagger= 1$.

### X Gate
An X gate maps $|0\rangle$ to $|1\rangle$ and $|1\rangle$ to $|0\rangle$. For classical computing, the NOT gate changes a 0 to a 1 and a 1 to a 0.

$$X = \begin{pmatrix}
0 & 1 \\
1 & 0 \\
\end{pmatrix}$$

### H Gate
A Hadamard gate maps the basis state $|0\rangle$ to $\frac{|0\rangle + |1\rangle}{\sqrt{2}}$ which is also written as $|+\rangle$ and $|1\rangle$ to $\frac{|0\rangle - |1\rangle}{\sqrt{2}}$ which is also written as $|-\rangle$. A measurement of the state $|+\rangle$ or of the state $|-\rangle$ will have equal probabilities of being $0$ or $1$, creating a superposition of states.

$$H = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1 \\
1 & -1 \\
\end{pmatrix}$$

## Two qubits quantum gates
A two qubits gate $U$ has a 4x4 unitary matrix form, $U^\dagger U=UU^\dagger= 1$.

### Controlled Not (CNOT, CX) Gate

If the first qubit is |1⟩ it performs the Pauli-X (NOT) operation on the second qubit, otherwise it leaves it unchanged

With the tensor ordering of qubits used in most physics textbooks:

$$CX = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \end{pmatrix}$$

With Qiskit tensor ordering of qubits:

$$CX = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
\end{pmatrix}$$

## Quantum Phase Estimation (QPE)
In quantum chemistry we need very accurate calculations of the total electronic energy of each molecule species involved in a chemical reaction [Burg]. The Quantum Phase Estimation (QPE) algorithm has a unique feature that it allows a bounded-error simulation of quantum systems which makes it one of the most promising application of future fault tolerant quantum computing. Given a unitary operator $U$, its eigenstate and eigenvalues, $U|\psi\rangle=e^{2 \pi i \theta}|\psi\rangle$, the ability to prepare a state $|\psi\rangle$ and the ability to apply $U$ itself, the QPE algorithm calculates $2^n θ$ where $n$ is the number of qubits used to estimate $\theta$ thereby allowing measurement of $\theta$ as precise as we want.

Recall that in Section 2.5, Postulate 5 – Time evolution dynamics, we saw that time evolution dynamics of a quantum system is described by the Schrödinger’s equation:

$$i \hbar \frac{d}{d t}|\psi\rangle=\hat{H}|\psi\rangle$$

For a time independent Hamiltonian $\hat{H}$ with initial condition$\left|\psi\left(t_{0}\right)\right\rangle$ the solution is:

$$|\psi(t)\rangle=U(t)\left|\psi\left(t_{0}\right)\right\rangle$$

where $U(t)=\exp \left(-i \frac{t}{\hbar} \hat{H}\right)$ is the unitary time-evolution operator. Further recall that any unitary matrix has eigenvalues of the form $e^{i \theta}$. An eigenvalue of $U(t)$ is also an eigenvalue of $\hat{H}$.

First we define a function $U(\theta)$ which creates a quantum circuit with a single qubit $\left|q_{0}\right\rangle$ and applies the following unitary:

$$U(\theta)\left|q_{0}\right\rangle=e^{2 \pi i \theta}\left|q_{0}\right\rangle=p(2 \pi \theta)\left|q_{0}\right\rangle$$

where $p(\lambda)$ is the gate which has the matrix form:

$$p(λ)= \begin{pmatrix}
1 & 0 \\
0 & e^{i\lambda} \\
\end{pmatrix}$$

### Define the function P_gate which returns the phase gate whose eigenvalue will be measured
Arguments:
- theta: the angle.
- endian: big_endian or little_endian

Returns:
- p: The matrix of the phase gate with angle $lambda = pi*2*theta$

In [10]:
function P_gate(theta::Float64, endian=big_endian)
    
    # Phase gate on the system to be measured qubit 0
    #p = Qgate.U(0.0, 0.0, pi*2*theta)
    
    lambda = theta
    p = [[1,0] [0, cos(lambda) + 1im*sin(lambda)]]
    
    return p
end

P_gate (generic function with 2 methods)

In [29]:
table = Matrix(undef, 5, 5)

5×5 Matrix{Any}:
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef

In [30]:
table[1,1] = 1

1

In [31]:
table[1,2] = P_gate

P_gate (generic function with 2 methods)

In [34]:
table[1,3] = "P_gate"

"P_gate"

In [35]:
table[1,5] = rand(3,3)

3×3 Matrix{Float64}:
 0.904279  0.146309  0.522186
 0.865622  0.138954  0.95615
 0.85688   0.711311  0.93522

In [38]:
table[1,5] = "extend"

"extend"

In [52]:
table[2,5] = "extend"

table1 = Matrix(undef, 5, 5)
table2 = Matrix(undef, 5, 5)

5×5 Matrix{Any}:
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef

In [53]:
[table1;table2]

UndefRefError: UndefRefError: access to undefined reference

In [60]:
table1 = Matrix(undef, 5, 5)

5×5 Matrix{Any}:
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef

In [61]:
A = [1 2 3 4 5]

1×5 Matrix{Int64}:
 1  2  3  4  5

In [57]:
table1 = zeros(5,5)

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [62]:
[table1;A]

UndefRefError: UndefRefError: access to undefined reference

In [59]:
table1[1,1] = "1"

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number
   @ Base twiceprecision.jl:273
  ...


In [11]:
PP = P_gate(0.25)
P_tensor = q_T2D(PP, 0, 2)

2×2 Matrix{ComplexF64}:
 1.0+0.0im       0.0+0.0im
 0.0+0.0im  0.968912+0.247404im

In [12]:
Phase_test = q.P(0.25)


2×2 Matrix{ComplexF64}:
 1.0+0.0im      -0.0-0.0im
 0.0+0.0im  0.968912+0.247404im

In [ ]:
gate1 = q.H # Hadamard gate
gate2 = q.I # Identity gate

### Define the function construct_circuit() which builds the $k^{th}$ iteration Quantum Phase Estimation circuit.

Arguments:
- u: The circuit representing the unitary operator whose eigenvalue (via phase) will be measured and one auxiliary qubit
- p: The unitary 2x2 matrix whose eigenvalue (via phase) will be measured
- k: the iteration index.
- omega: the feedback angle.
- verbose: set to true to activate trace
- endian: big_endian or little_endian

Returns:
- u: the quantum circuit per iteration

In [17]:
# step 1
H = q.H
H = q_T2D(H, 1, 2) # H gate on the auxiliary qubit 1


4×4 Matrix{Float64}:
 0.707107  0.0        0.707107   0.0
 0.0       0.707107   0.0        0.707107
 0.707107  0.0       -0.707107  -0.0
 0.0       0.707107  -0.0       -0.707107

In [18]:
display(H)

4×4 Matrix{Float64}:
 0.707107  0.0        0.707107   0.0
 0.0       0.707107   0.0        0.707107
 0.707107  0.0       -0.707107  -0.0
 0.0       0.707107  -0.0       -0.707107

In [19]:
function construct_circuit(u::Main.quantum_circuit.qc_initstruct, p::Matrix{ComplexF64}, k::Int64, omega::Float64; 
        verbose=false, endian=big_endian)
    
    if verbose
        println("\nconstruct_circuit k: ", k)
    end
    
    H = Qgate.H
    H = Qgate_T2D(H, 1, 2, endian) # H gate on the auxiliary qubit 1
    display(H)
    # Hadamard on the auxiliary qubit 1
    u = op(u, H)
    
    # Compute px = p to the power 2^(k-1)
    r = 2^(k-1)
    if verbose
        println("r: ",r)
    end
    
    II::Array{Float64,2} = Matrix(I,2,2)
    px = II
    
    for i in 1:r
        px = px * p
    end
    
    if verbose
        println("\nconstruct_circuit px")
        show(stdout, "text/plain", px)
        println(" ")
    end
    
    # Apply the controlled gate of px
    C_U = Qgate.ctrl_gate(px)
    u = op(u, C_U)
    # Apply phase gate p with angle omega on the auxiliary qubit 1
    #po = Qgate.U(0.0, 0.0, omega)
    po = [[1,0] [0, cos(omega) + 1im*sin(omega)]]
    po = Qgate_T2D(po, 1, 2, endian) # Phase gate on the auxiliary qubit 1
    display(po)

    u = op(u, po)
    
    # Hadamard on the auxiliary qubit 1
    u = op(u, H)
    
    if verbose
        #show(stdout, "text/plain", u)
        println("\nconstruct_circuit")
        show(stdout, "text/plain", u)
        println(" ")
        println("u.state_vector: ", u.state_vector)
    end
    
    return u
end

construct_circuit (generic function with 1 method)

### Define the function estimate() which iteratively estimates the eigenphase of the input unitary and initial-state

Arguments:
- n_iter: The number of iterations
- p: The unitary 2x2 matrix whose eigenvalue (via phase) will be measured
- verbose: set to true to activate trace
- endian: big_endian or little_endian

Returns:
- Estimated phase

In [20]:
function estimate(n_iter::Int64, p::Matrix{ComplexF64}; verbose=false, endian=big_endian)
    
    v = qc_init(2; big_endian=endian, c_sv=nothing, err_tol=1e-14, show_matrix=true)
    
    # Apply phase gate on the system to be measured qubit 0
    p2 = Qgate_T2D(p, 0, 2, endian)
    v = op(v, p2)
    
    # Apply X gate on the system to be measured qubit 0
    X = Qgate.X
    X = Qgate_T2D(X, 0, 2, endian)
    v = op(v, X)
    
    #println("\nestimate - matrix v: ")
    #show(stdout, "text/plain", v)
    
    omega_coef = 0
    
    for k in n_iter:-1:1
        omega_coef /= 2
        
        u = construct_circuit(v, p, k, -2*pi*omega_coef; verbose=verbose)
        
        # Simulate measurement of the auxiliary qubit 1
        prob_01 = abs2(u.state_vector[3])
        prob_00 = abs2(u.state_vector[1])
        
        x = (prob_01 > prob_00 || prob_00 - prob_01 < 1e-7)
        
        if x
            omega_coef = omega_coef + 1/2
        end
        
        if verbose
            println("\nestimate - k: ", k, " measurement: ", x, " omega_coef: ", omega_coef)
        end
        
    end
    
    return omega_coef
end

estimate (generic function with 1 method)

### Run the iterative phase estimation

In [21]:
theta = 0.0
p = P_gate(theta)

2×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

In [ ]:
estimate(0, p, verbose=true)

In [ ]:
theta = 1/4
p = P_gate(theta)

In [ ]:
P2 = 

In [ ]:
estimate(1, p, verbose=true)

In [ ]:
theta = 1/2
p = P_gate(theta)

In [ ]:
estimate(1, p)

In [ ]:
theta = 1/2 + 1/4
p = P_gate(theta)

In [ ]:
estimate(2, p)

In [ ]:
theta = 1/2 + 1/8
p = P_gate(theta)

In [ ]:
estimate(2, p)

In [ ]:
theta = 1/2 + 1/4 + 1/8
p = P_gate(theta)

In [ ]:
estimate(5, p)

In [ ]:
theta = 1/2 + 1/4 + 1/8 + 1/16
p = P_gate(theta)

In [ ]:
estimate(5, p)

In [ ]:
theta = 1/2 + 1/4 + 1/8 + 1/16 + 1/32
p = P_gate(theta)

In [ ]:
estimate(5, p)

In [ ]:
theta = 1/2 + 1/4 + 1/8 + 1/16 + 1/32 + 1/64 + 1/128
p = P_gate(theta)

In [ ]:
estimate(5, p)

In [ ]:
estimate(7, p)

## References

[Groenenboom 1] Prof. Gerrit C. Groenenboom, Quantum theoretical chemistry (NWI-MOL112), May 26, 2023, https://www.theochem.ru.nl/ctc2/pdf/qtclecture.pdf

[Groenenboom 2] Prof. Gerrit C. Groenenboom, Computational and Theoretical Chemistry 2, NWI-MOL176 (3EC),https://www.theochem.ru.nl/ctc2/

[Sharkey] Sharkey, Keeper L., and Chancé, Alain. 2022. Quantum Chemistry and Computing for the Curious: Illustrated with Python and Qiskit Code. Packt, Pub. ISBN-13: 978-1803243900

[IPE] IterativePhaseEstimation, https://qiskit.org/ecosystem/algorithms/stubs/qiskit_algorithms.IterativePhaseEstimation.html

[IPE_Source] Source code for qiskit.algorithms.phase_estimators.ipe,
https://qiskit.org/documentation/stable/0.41/_modules/qiskit/algorithms/phase_estimators/ipe.html#IterativePhaseEstimation

[lab3] ibm-quantum-challenge-spring-2023, lab3.ipynb, https://github.com/qiskit-community/ibm-quantum-challenge-spring-2023/blob/main/content/lab_3/lab3.ipynb

[IPE_video] Iterative Quantum Phase Estimation | Qiskit Global Summer School 2023, https://www.youtube.com/watch?v=aLSM0_H8hUE

[DavitKhach] Iterative quantum phase estimation, https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/iterative_quantum_phase_estimation.ipynb

[Devesh] Implementing Quantum Phase Estimation Algorithm using Qiskit, https://medium.com/@deveshq/implementing-quantum-phase-estimation-algorithm-using-qiskit-e808e8167d32

## Display Julia version information

In [ ]:
# Display Julia version information
versioninfo()